In [2]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristcs OLS

Professor said we should start with a regular OLS Regression.
I quickly looked into this and think a random effects panel regression would probably fit our data better (still need to read more into this)
So probably we can first do a regular OLS and than compare it to the random effects panel regression. Just some initial thoughts though!!!
Here is some intial code to run them:


In [3]:
#decide which df to use
df = pd.read_csv("Dataframes/characteristics.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]

df

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Net Income after Tax
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,189965000.0
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,307205000.0
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,179203000.0
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,107542000.0
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,184573000.0
...,...,...,...,...,...
20115,AVY.N,2014-01-01,1.471,1.583900e+09,42100000.0
20116,AVY.N,2013-10-01,8.095,1.504900e+09,62000000.0
20117,AVY.N,2013-07-01,1.068,1.552300e+09,70800000.0
20118,AVY.N,2013-04-01,2.482,1.498900e+09,66800000.0


#### OLS Regression

In [4]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new.isna().sum()


Instrument                              0
Date                                    0
Earnings Per Share - Actual Surprise    0
Revenue - Actual                        0
Net Income after Tax                    0
dtype: int64

In [5]:
df_accuracy_new

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Net Income after Tax
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,189965000.0
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,307205000.0
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,179203000.0
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,107542000.0
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,184573000.0
...,...,...,...,...,...
20115,AVY.N,2014-01-01,1.471,1.583900e+09,42100000.0
20116,AVY.N,2013-10-01,8.095,1.504900e+09,62000000.0
20117,AVY.N,2013-07-01,1.068,1.552300e+09,70800000.0
20118,AVY.N,2013-04-01,2.482,1.498900e+09,66800000.0


In [6]:


rescale = df_accuracy_new
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Net Income after Tax"] = MinMaxScaler().fit_transform(np.array(rescale["Net Income after Tax"]).reshape(-1,1))

rescale


,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Net Income after Tax
0,POOL.OQ,2022-10-01,4.196,0.065637,0.556805
1,POOL.OQ,2022-07-01,1.503,0.068358,0.558114
2,POOL.OQ,2022-04-01,34.342,0.064385,0.556685
3,POOL.OQ,2022-01-01,40.267,0.062055,0.555885
4,POOL.OQ,2021-10-01,17.194,0.064377,0.556745
...,...,...,...,...,...
20115,AVY.N,2014-01-01,1.471,0.065443,0.555155
20116,AVY.N,2013-10-01,8.095,0.064955,0.555377
20117,AVY.N,2013-07-01,1.068,0.065247,0.555475
20118,AVY.N,2013-04-01,2.482,0.064917,0.555431


In [7]:
y = rescale["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = rescale.iloc[:, 3:]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.001
Model:                                              OLS   Adj. R-squared:                  0.001
Method:                                   Least Squares   F-statistic:                     11.73
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):           8.10e-06
Time:                                          16:55:07   Log-Likelihood:                -79225.
No. Observations:                                 18603   AIC:                         1.585e+05
Df Residuals:                                     18600   BIC:                         1.585e+05
Df Model:                                             2                                         
Covariance Type:                              nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  -12.2582      4.224     -2.902      0.004     -20.537      -3.979
Revenue - Actual        -1.9242      2.041     -0.943      0.346      -5.925       2.076
Net Income after Tax    34.6555      7.698      4.502      0.000      19.568      49.743
==============================================================================
Omnibus:                     3775.143   Durbin-Watson:                   1.627
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33769.713
Skew:                           0.722   Prob(JB):                         0.00
Kurtosis:                       9.440   Cond. No.                         81.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
y = df_accuracy_new["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = df_accuracy_new.iloc[:, 3:]

scaler = StandardScaler()
X = scaler.fit_transform(X)

reg= LinearRegression().fit(X,y)

In [9]:
reg

LinearRegression()

#### Random Effects Panel Regression

In [10]:
df_accuracy_new['Date'] = pd.to_datetime(df_accuracy_new['Date'], infer_datetime_format=True).dt.tz_localize(None)
df_accuracy_new.dtypes

Instrument                                      object
Date                                    datetime64[ns]
Earnings Per Share - Actual Surprise           float64
Revenue - Actual                               float64
Net Income after Tax                           float64
dtype: object

In [11]:
df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

Earnings Per Share - Actual Surprise  Revenue - Actual  \
Instrument Date                                                                 
POOL.OQ    2022-10-01                                 4.196          0.065637   
           2022-07-01                                 1.503          0.068358   
           2022-04-01                                34.342          0.064385   
           2022-01-01                                40.267          0.062055   
           2021-10-01                                17.194          0.064377   
...                                                     ...               ...   
AVY.N      2014-01-01                                 1.471          0.065443   
           2013-10-01                                 8.095          0.064955   
           2013-07-01                                 1.068          0.065247   
           2013-04-01                                 2.482          0.064917   
           2013-01-01                                11.178          0.065123   

                       Net Income after Tax  
Instrument Date                              
POOL.OQ    2022-10-01              0.556805  
           2022-07-01              0.558114  
           2022-04-01              0.556685  
           2022-01-01              0.555885  
           2021-10-01              0.556745  
...                                     ...  
AVY.N      2014-01-01              0.555155  
           2013-10-01              0.555377  
           2013-07-01              0.555475  
           2013-04-01              0.555431  
           2013-01-01              0.555058  

[18603 rows x 3 columns]

In [12]:
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as tsap
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_white
from matplotlib import pyplot as plt

In [13]:
#decide which df to use
df = pd.read_csv("Dataframes/economic_story_dataframe.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]

df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Number of Analysts,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,"WACC Inflation Adjusted Risk Free Rate, (%)",Net Income after Tax,Price Target - Mean,Price Close
0,POOL.OQ,2022-10-01,4.78,4.58750,4.196,1.615339e+09,1.406160e+10,10.0,2.27273,2.200,3.747421,189965000.0,360.11111,318.21
1,POOL.OQ,2022-07-01,7.63,7.51700,1.503,2.055818e+09,1.556187e+10,9.0,2.20000,2.000,3.092855,307205000.0,435.37500,351.23
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,1.412650e+09,1.843682e+10,9.0,2.00000,2.300,2.325202,179203000.0,519.88889,422.85
3,POOL.OQ,2022-01-01,2.63,1.87500,40.267,1.035557e+09,2.384882e+10,8.0,2.30000,2.300,1.515266,107542000.0,571.00000,566.00
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,1.411448e+09,1.769954e+10,8.0,2.30000,2.300,1.527139,184573000.0,545.28571,434.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,0.69,0.68000,1.471,1.583900e+09,5.553068e+09,8.0,2.33333,2.625,NaN,42100000.0,54.62500,50.19
20116,AVY.N,2013-10-01,0.69,0.63833,8.095,1.504900e+09,5.029514e+09,7.0,2.62500,2.875,NaN,62000000.0,47.71429,43.52
20117,AVY.N,2013-07-01,0.71,0.70250,1.068,1.552300e+09,5.436229e+09,8.0,2.87500,2.875,NaN,70800000.0,45.00000,42.76
20118,AVY.N,2013-04-01,0.59,0.57571,2.482,1.498900e+09,5.459128e+09,7.0,2.87500,3.000,NaN,66800000.0,42.57143,43.07


In [14]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new.isna().sum()


Instrument                                     0
Date                                           0
Earnings Per Share - Actual                    0
Earnings Per Share - Mean                      0
Earnings Per Share - Actual Surprise           0
Revenue - Actual                               0
Enterprise Value                               0
Number of Analysts                             0
Recommendation - Mean (1-5)                    0
Recommendation - Mean (1-5).1                  0
WACC Inflation Adjusted Risk Free Rate, (%)    0
Net Income after Tax                           0
Price Target - Mean                            0
Price Close                                    0
dtype: int64

In [15]:
y_var_name = 'Earnings Per Share - Actual Surprise'
X_var_names = ['Revenue - Actual', "Net Income after Tax", "Number of Analysts", "Recommendation - Mean (1-5)", "WACC Inflation Adjusted Risk Free Rate, (%)", "Net Income after Tax", "Price Target - Mean", "Price Close"]
pooled_y=df_accuracy_new[y_var_name]
pooled_X=df_accuracy_new[X_var_names]

In [16]:
pooled_X = sm.add_constant(pooled_X)

In [17]:
pooled_olsr_model = sm.OLS(endog=pooled_y, exog=pooled_X)

In [18]:
pooled_olsr_model_results = pooled_olsr_model.fit()

In [19]:
print(pooled_olsr_model_results.summary())

                                     OLS Regression Results                                     
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.017
Model:                                              OLS   Adj. R-squared:                  0.017
Method:                                   Least Squares   F-statistic:                     30.54
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):           3.36e-42
Time:                                          16:55:07   Log-Likelihood:                -51361.
No. Observations:                                 12040   AIC:                         1.027e+05
Df Residuals:                                     12032   BIC:                         1.028e+05
Df Model:                                             7                                         
Covariance Type:                              nonrobust                                         
                              

In [20]:
y = df_accuracy_new["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = df_accuracy_new.iloc[:, 5:]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.019
Model:                                              OLS   Adj. R-squared:                  0.018
Method:                                   Least Squares   F-statistic:                     26.07
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):           5.06e-45
Time:                                          16:55:07   Log-Likelihood:                -51351.
No. Observations:                                 12040   AIC:                         1.027e+05
Df Residuals:                                     12030   BIC:                         1.028e+05
Df Model:                                             9                                         
Covariance Type:                              nonrobust                                         
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
const                                          15.4473      1.236     12.495      0.000      13.024      17.871
Revenue - Actual                             2.281e-12   1.58e-11      0.144      0.886   -2.88e-11    3.33e-11
Enterprise Value                            -1.892e-12   1.73e-12     -1.095      0.274   -5.28e-12     1.5e-12
Number of Analysts                              0.0201      0.025      0.817      0.414      -0.028       0.068
Recommendation - Mean (1-5)                    -6.8331      1.271     -5.375      0.000      -9.325      -4.341
Recommendation - Mean (1-5).1                   5.5800      1.271      4.391      0.000       3.089       8.071
WACC Inflation Adjusted Risk Free Rate, (%)    -2.7187      0.206    -13.169      0.000      -3.123      -2.314
Net Income after Tax                         4.115e-10   1.14e-10      3.603      0.000    1.88e-10    6.35e-10
Price Target - Mean                            -0.0025      0.005     -0.472      0.637      -0.013       0.008
Price Close                                     0.0037      0.006      0.600      0.548      -0.008       0.016
==============================================================================
Omnibus:                     2126.783   Durbin-Watson:                   1.633
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18860.390
Skew:                           0.592   Prob(JB):                         0.00
Kurtosis:                       9.016   Cond. No.                     1.81e+12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.81e+12. This might indicate that there are
strong multicollinearity or other numerical problems.
"""